# JIRA TICKETING AUTOMATION

## A. API Get Jira Request fields labels, fields responses, 

In [ ]:
#Import necessary libraries

import requests
from requests.auth import HTTPBasicAuth
import json
import os
from dotenv import load_dotenv
import psycopg2 #to connect to PostgreSQL database
from psycopg2.extras import execute_values #a function to insert multiple records into the database
import logging
import psycopg2.extras

In [2]:
#load_dotenv() variables and secrets
load_dotenv()
admin_email = os.getenv("admin_email")  
API_TOKEN =os.getenv("API_TOKEN") 
FIELDVALUE_API=os.getenv("FIELDVALUE_API")
JIRA_URL= os.getenv("JIRA_URL") 
#"https://declaunchpad.atlassian.net" #
REQUESTAPI_TOKEN=os.getenv("REQUESTAPI_TOKEN")
CLOUD_ID= os.getenv("CLOUD_ID") 
#C_ID=os.getenv("C_ID")
REQUESTTYPEID = os.getenv("REQUESTTYPEID") 
SERVICEDESKID = os.getenv("SERVICEDESKID") 
FORMTEMPLATE_ID = os.getenv("FORMTEMPLATE_ID")

logging.basicConfig(
    filename="jira_automation.log",
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s",
)




### A1. Fetching the fields required to create customer request:

These fields are basically the Summary and Description field called "What are the details of your request?" in the ticket

In [3]:
#Get all the field question from Jira request form

url = f"{JIRA_URL}/rest/servicedeskapi/servicedesk/{SERVICEDESKID}/requesttype/{REQUESTTYPEID}/field"
#url

auth = HTTPBasicAuth(admin_email,REQUESTAPI_TOKEN)
headers = {
"Authorization": "Basic <base64 encoded email:token>",
"Accept": "application/json"
}

response = requests.get(url, headers=headers, auth=auth)

requestform = json.dumps(json.loads(response.text), sort_keys=True, indent=4, separators=(",", ": "))
requestform



'{\n    "canAddRequestParticipants": true,\n    "canRaiseOnBehalfOf": true,\n    "requestTypeFields": [\n        {\n            "defaultValues": [],\n            "description": "",\n            "fieldId": "summary",\n            "jiraSchema": {\n                "system": "summary",\n                "type": "string"\n            },\n            "name": "Summary",\n            "required": true,\n            "validValues": [],\n            "visible": true\n        },\n        {\n            "defaultValues": [],\n            "description": "",\n            "fieldId": "description",\n            "jiraSchema": {\n                "system": "description",\n                "type": "string"\n            },\n            "name": "What are the details of your request?",\n            "required": true,\n            "validValues": [],\n            "visible": true\n        }\n    ]\n}'

### A2. Getting the form fields linked to the request type:
**These fields are called the options fields where you have dropdown to select from.**

**This would return the form structure, the questions ID and the options under the drop down questions.**
**Also, the form template Id would be derived from the output of the script**

In [4]:
# Get all field value in the dropdown type of questions
url = f'https://api.atlassian.com/jira/forms/cloud/{CLOUD_ID}/servicedesk/{SERVICEDESKID}/requesttype/{REQUESTTYPEID}/form'

auth = HTTPBasicAuth(admin_email, FIELDVALUE_API)
#url

headers = {
"Authorization": "Basic <base64 encoded email:token>",
"Accept": "application/json"
}

response = requests.get(url, headers=headers, auth=auth)

formfield = response.json()

with open('form_request.json', 'w', encoding='utf-8') as file:
    json.dump(formfield, file, ensure_ascii=False, indent=4)

#formfield = json.dumps(json.loads(response.text), sort_keys=True, indent=4, separators=(",", ": "))
#formfield


### A3. Fetching manually created customer request responses

**This is an example of a ticket created manually and the response for the requestform which would have the Summary and Description answer (What are the details of your request?).**

In [5]:
# Get all field value in the dropdown type of questions
url = f'{JIRA_URL}/rest/servicedeskapi/request'

auth = HTTPBasicAuth(admin_email, FIELDVALUE_API)
#url

#headers = {
#"Authorization": "Basic <base64 encoded email:token>",
#"Accept": "application/json"
#}

headers = {
  "Accept": "application/json",
  "Content-Type": "application/json",
  "Authorization": "Bearer <access_token>"
}

#data=json.dumps({})

response = requests.get(url, 
                         #data = data
                         headers=headers, auth=auth)

ticket= json.dumps(json.loads(response.text), sort_keys=True, indent=4, separators=(",", ": "))
ticket


'{\n    "_expands": [\n        "participant",\n        "status",\n        "sla",\n        "requestType",\n        "serviceDesk",\n        "attachment",\n        "action",\n        "comment"\n    ],\n    "_links": {\n        "base": "https://declaunchpad.atlassian.net",\n        "context": "",\n        "self": "https://declaunchpad.atlassian.net/rest/servicedeskapi/request"\n    },\n    "isLastPage": true,\n    "limit": 50,\n    "size": 1,\n    "start": 0,\n    "values": [\n        {\n            "_expands": [\n                "participant",\n                "status",\n                "sla",\n                "requestType",\n                "serviceDesk",\n                "attachment",\n                "action",\n                "comment"\n            ],\n            "_links": {\n                "agent": "https://declaunchpad.atlassian.net/browse/CCX-1",\n                "jiraRest": "https://declaunchpad.atlassian.net/rest/api/2/issue/10033",\n                "self": "https://declaunchpa

To get the Form template ID of the Issue, the get issue id API was used

In [6]:
# Get the form template id
url = f'https://api.atlassian.com/jira/forms/cloud/{CLOUD_ID}/issue/CCX-1/form'

auth = HTTPBasicAuth(admin_email, FIELDVALUE_API)
#url

headers = {
  "Accept": "application/json",
  "Content-Type": "application/json",
  "Authorization": "Bearer <access_token>"
}

response = requests.get(url, 
                         #data = data
                         headers=headers, auth=auth)

print(json.dumps(json.loads(response.text), sort_keys=True, indent=4, separators=(",", ": ")))

[
    {
        "formTemplate": {
            "id": "336bb096-c808-465b-a5f1-c4fb2de738d5"
        },
        "id": "5f60ce33-1fdd-424a-8213-049c4f8a6f9a",
        "internal": false,
        "lock": false,
        "name": "Phone equipment order",
        "submitted": true,
        "updated": "2025-11-10T11:01:53.931Z"
    }
]


### API Get Script to get the field responses for a filled request

In [7]:
# Get all field responses in the dropdown type of questions
url = f'https://api.atlassian.com/jira/forms/cloud/{CLOUD_ID}/request/CCX-1/form/5f60ce33-1fdd-424a-8213-049c4f8a6f9a/format/answers'


auth = HTTPBasicAuth(admin_email, FIELDVALUE_API)
#url

headers = {
"Authorization": "Basic <base64 encoded email:token>",
"Accept": "application/json"
}

# headers = {
#   "Accept": "application/json",
#   "Content-Type": "application/json",
#   "Authorization": "Bearer <access_token>"
# }

#data=json.dumps({})

response = requests.get(url, 
                         #data = data
                         headers=headers, auth=auth)

responses = json.dumps(json.loads(response.text), sort_keys=True, indent=4, separators=(",", ": "))
responses

'[\n    {\n        "answer": "adeboladesoyin@gmail.com",\n        "label": " Email"\n    },\n    {\n        "answer": "Thank you.",\n        "label": "Comments"\n    },\n    {\n        "answer": "ADEBOLA ADESOYIN",\n        "label": "Name"\n    },\n    {\n        "answer": "08169567636",\n        "label": "Phone"\n    },\n    {\n        "answer": "IT",\n        "label": "Department"\n    },\n    {\n        "answer": "Data",\n        "label": "Job title"\n    },\n    {\n        "answer": "",\n        "label": "Cost center (acct #)"\n    },\n    {\n        "answer": "2025-11-11",\n        "label": "Date needed by"\n    },\n    {\n        "answer": "Mobile phone",\n        "choice": "167",\n        "label": "Equipment requested"\n    },\n    {\n        "answer": "Relocate existing to a new location ",\n        "choice": "182",\n        "label": "Installation requested"\n    },\n    {\n        "answer": "Permanent use",\n        "choice": "184",\n        "label": "This equipment is for "\n

In [8]:
load_dotenv()
SB_USER = os.getenv("SB_USER")
SB_PASSWORD = os.getenv("SB_PASSWORD")
SB_HOST = os.getenv("SB_HOST")
SB_PORT = os.getenv("SB_PORT")
SB_NAME = os.getenv("SB_NAME")


DB_USER = os.getenv("SB_USER")
DB_PASSWORD = os.getenv("SB_PASSWORD")
DB_HOST = os.getenv("SB_HOST")
DB_PORT = os.getenv("SB_PORT")
DB_NAME = os.getenv("SB_NAME")

# Get the supabase DB credential for connection
def get_connection():
    return psycopg2.connect(
        user=SB_USER,
        password=SB_PASSWORD,
        host=SB_HOST,
        port=SB_PORT,
        dbname=SB_NAME 
    )

# Get the log table DB credential for connection
def get_connection2():
    return psycopg2.connect(
        user=DB_USER,
        password=DB_PASSWORD,
        host=DB_HOST,
        port=DB_PORT,
        dbname=DB_NAME 
    )


In [ ]:
def get_users(customers):
    try:
        with get_connection() as conn:
            with conn.cursor(cursor_factory=psycopg2.extras.RealDictCursor) as cur:
                query = """
                    SELECT *
                    FROM phonerequest p
                    WHERE p.emailaddress = ANY(%s);
                """
                cur.execute(query, (customers,))
                results = cur.fetchall()
        return results
    except Exception as e:
        print(f"Error fetching users: {e}")
        return []


In [10]:
# #function to retrieve user details (emailaddress)

# def retrieve_user_detail(email):
#     try:
#         with get_connection() as conn: 
#             with conn.cursor() as cur:
#                 cur.execute("SELECT * FROM phonerequest WHERE emailaddress = %s", (email,))
#                 a= cur.fetchall()
#                 print(a)
#     except Exception as e:
#         print(f'error')
# retrieve_user_detail('jared.wood@example.com')


In [11]:
# Function to check log table in postgres DB
def is_logged(email):
    try:
        with get_connection2() as conn: 
            with conn.cursor() as cur:
                cur.execute("SELECT 1 FROM integration_ticket_log WHERE email = %s", (email,))
                return cur.fetchone() is not None
    except Exception as e:
        print("Error checking log table:", e)
        return False
    

# Function to log the ticket status in the log tble

def log_ticket(name, issue_key, email):
    try:
        with get_connection2() as conn:
            with conn.cursor() as cur:
                cur.execute("""
                    INSERT INTO integration_ticket_log
                    (name, issue_key, email, is_ticket_created, is_form_updated)
                    VALUES (%s, %s, %s, 'Yes', 'No')
                    ON CONFLICT (email) DO NOTHING;
                """, (name, issue_key, email))
                conn.commit()
    except Exception as e:
        print("Error logging ticket:", e)

In [ ]:

#Getting the user data mapped to a variable
def record_process(record, records_proc):
    skipped = 0
    success = 0
    failed = 0
    skipped = 0
    hash = set()

    for users in record:
        try:
        # Get hash for new record
            #hash_record = get_users(users)
            hash_record = users['emailaddress']
            if hash_record in hash:
                logging.debug(f'already processed for {users ['newusername']}')
                skipped +=1
                continue


            #Getting the request data
            name = users['newusername']
            job = users['job']
            phone = users['phonenumber']
            email = users['emailaddress']
            department = users['departmentname']
            cost_center = users['costcenter']
            installation_type = users['telephonelinesandinstallations']
            # Seperating the "Cordless handset; Mobile phone; Smartphone" using delimiter
            equipment = [item.strip() for item in users['handsetsandheadsets'].split(";")]
            time_frame = users['timeframe']
            ending_date = users['dateneededby']
            comments = users['Comments']

            # Since we have a branching in timeframe question, if temporary, the approximateendingdate will pop up.Hence, an if statement would be required here

            # declare variable
            approximate_endingdate = None

            # Now, let's make the consitional statement if Temporary use (three months or less) is selected as option
            if(time_frame=="Temporary use (three months or less')"):
                time_usageid=['183']
                approximate_endingdate = users['approximateendingdate']
            elif(time_frame=='Permanent use'):
                time_usageid=['184']

            # creating a dictionary to store the fieldid of each options for the checkbox type of questions
            installationoption = {
                "New extension including new cabling and socket": ['160'],
                "New extension to an existing, inactive socket": ['161'],
                "Relocate existing to a new location": ['182'],
                "Convert existing extension from analogue to digital": ['190'],
                "Relocate an existing extension to an existing inactive, socket": ['191'],
                "Swap of telephone extensions": ['192'],
                "Other... (multi-line hunt group setup)": ['0']
            }
            installation_id=installationoption[installation_type]

            # Let's do same for Equipment requested which is also a checkbox field by mapping the field value to the field id, 
            #This is beacuse, the value that the syatem will pick is the id and not the label itself.
            equipmentoption=[]
            equipmentmap={
                "Handset speaker phone": ['164'],
                "Cordless headset": ['165'],
                "Cordless handset": ['166'],
                "Mobile phone": ['167'],
                "Smartphone": ['168'],
                "SIM card only": ['194'],
                "Other... (spare adapter or special desk mount); SIM card only; Mobile phone": ['0']
            }
            for options in equipment:
                   if options in equipmentmap:
                          equipmentoption.append(equipmentmap[options])

            ticket = {
                # The first aspect of the Jira form   
                "serviceDeskId": SERVICEDESKID,
                "requestTypeId": REQUESTTYPEID,
                "requestFieldValues": {
                    "summary": f"New request from {name}",
                    "description": f"Request submitted by {name} using ({email})"
                },
                #The section called "person making request"
                "form":{
                       "answers": {
                              "199": {"text": name},
                              "202": {"text": email},
                              "201": {"text": phone},
                              "200": {"text": job},
                              "203": {"text":department},
                              "204": {"text":cost_center},

                              #Telephone lines and installations question
                              "157": {"choices": installation_id},

                              #Handsets and Headsets
                              "159": {"choices": equipmentoption},

                              #Timeframe
                              "205": {"choices": time_frame},
                              "206": {"choices": ending_date},
                              #"197": {"choices": approximate_endingdate},

                              #Comments question
                              "189": {"text": comments}                              
                       }
                }
            }

            #Now, we can add approximate_endingdate if the temporary use is selected.
            if approximate_endingdate:
                   ticket["form"]["answers"]["197"] = {"date": approximate_endingdate}

            #After filling the form, submit the request
            response = requests.post(
            f"{JIRA_URL}/rest/servicedeskapi/request",
            headers=headers,
            auth=auth,
            data=json.dumps(ticket),
            timeout=30
        )

            if response.status_code in [200,201]:
                issue_key = response.json().get("issueKey")
                print(f"Ticket created: {issue_key}")
                logging.info("============================================")
                logging.info(f'Ticket successfullly created for {name} with {issue_key}')
                success +=1
                hash.add(hash_record)
                log_ticket(name, issue_key, email)
            else:
                logging.error(f'Failure to create ticket for {name}: Status {response.status_code}')
                print(f"Failed to create ticket for {email}: {response.status_code}, {response.text}")
                failed += 1
        except KeyError as k:
             # for any missing fields
             logging.error(f'There are some missing fields for user {k}', exc_info=True)
             failed +=1
            
        except requests.exceptions.RequestException as e:
            logging.error(f"Request exception for {users.get('emailaddress', 'unknown')}: {e}", exc_info=True)
            failed += 1

    return {"success": success, "failed": failed, "skipped": skipped}
            

                   


In [18]:
#create_jira_ticket(retrieve_user_detail('jared.wood@example.com'))
#retrieve_user_detail('jared.wood@example.com')
emails = ['jared.wood@example.com', 'terri.hunt@example.com']
records = get_users(emails)
records_proc = []
result = record_process(records, records_proc)
print(result)




Error fetching users: can't adapt type 'RealDictRow'


TypeError: unhashable type: 'list'

In [ ]:
import psycopg2.extras
import requests
import json
import logging

# Assuming get_connection(), SERVICEDESKID, REQUESTTYPEID, JIRA_URL, headers, and auth are defined elsewhere

def get_users(customers):
    """Fetch user data from the phonerequest table by email list."""
    try:
        with get_connection() as conn:
            with conn.cursor(cursor_factory=psycopg2.extras.RealDictCursor) as cur:
                query = """
                    SELECT *
                    FROM phonerequest p
                    WHERE p.emailaddress = ANY(%s);
                """
                cur.execute(query, (customers,))
                results = cur.fetchall()
        return results
    except Exception as e:
        print(f"Error fetching users: {e}")
        return []


def record_process(records, records_proc):
    """Process each user record and create Jira tickets."""
    success = 0
    failed = 0
    skipped = 0
    processed_hash = set()

    for user in records:
        try:
            # Avoid duplicates
            user_key = user['emailaddress']
            if user_key in processed_hash:
                logging.debug(f"Already processed for {user['newusername']}")
                skipped += 1
                continue

            # --- Extract request data ---
            name = user['newusername']
            job = user['job']
            phone = user['phonenumber']
            email = user['emailaddress']
            department = user['departmentname']
            cost_center = user['costcenter']
            installation_type = user['telephonelinesandinstallations']
            equipment_list = [item.strip() for item in user['handsetsandheadsets'].split(";")]
            time_frame = user['timeframe']
            ending_date = user['dateneededby']
            comments = user['comments']  # changed to lowercase (PostgreSQL returns lowercase)
            approximate_endingdate = None

            # --- Conditional timeframe handling ---
            if time_frame == "Temporary use (three months or less)":
                time_usage_id = ['183']
                approximate_endingdate = user.get('approximateendingdate')
            elif time_frame == "Permanent use":
                time_usage_id = ['184']
            else:
                time_usage_id = []

            # --- Installation options mapping ---
            installation_map = {
                "New extension including new cabling and socket": ['160'],
                "New extension to an existing, inactive socket": ['161'],
                "Relocate existing to a new location": ['182'],
                "Convert existing extension from analogue to digital": ['190'],
                "Relocate an existing extension to an existing inactive, socket": ['191'],
                "Swap of telephone extensions": ['192'],
                "Other... (multi-line hunt group setup)": ['0']
            }
            installation_id = installation_map.get(installation_type, ['0'])

            # --- Equipment mapping ---
            equipment_map = {
                "Handset speaker phone": ['164'],
                "Cordless headset": ['165'],
                "Cordless handset": ['166'],
                "Mobile phone": ['167'],
                "Smartphone": ['168'],
                "SIM card only": ['194'],
                "Other... (spare adapter or special desk mount); SIM card only; Mobile phone": ['0']
            }

            equipment_options = []
            for eq in equipment_list:
                if eq in equipment_map:
                    equipment_options.append(equipment_map[eq])

            # --- Construct the JIRA ticket payload ---
            ticket = {
                "serviceDeskId": SERVICEDESKID,
                "requestTypeId": REQUESTTYPEID,
                "requestFieldValues": {
                    "summary": f"New request from {name}",
                    "description": f"Request submitted by {name} using ({email})"
                },
                "form": {
                    "answers": {
                        "199": {"text": name},
                        "202": {"text": email},
                        "201": {"text": phone},
                        "200": {"text": job},
                        "203": {"text": department},
                        "204": {"text": cost_center},
                        "157": {"choices": installation_id},
                        "159": {"choices": equipment_options},
                        "205": {"choices": time_usage_id},
                        "206": {"date": ending_date},
                        "189": {"text": comments}
                    }
                }
            }

            if approximate_endingdate:
                ticket["form"]["answers"]["197"] = {"date": approximate_endingdate}

            # --- Submit the ticket to Jira ---
            response = requests.post(
                f"{JIRA_URL}/rest/servicedeskapi/request",
                headers=headers,
                auth=auth,
                data=json.dumps(ticket),
                timeout=30
            )

            if response.status_code in [200, 201]:
                issue_key = response.json().get("issueKey")
                print(f"Ticket created: {issue_key}")
                logging.info(f"Ticket successfully created for {name} with {issue_key}")
                success += 1
                processed_hash.add(user_key)
                log_ticket(name, issue_key, email)
            else:
                logging.error(f"Failed to create ticket for {name}: Status {response.status_code}")
                print(f"Failed to create ticket for {email}: {response.status_code}, {response.text}")
                failed += 1

        except KeyError as k:
            logging.error(f"Missing field {k} for user {user.get('emailaddress', 'unknown')}", exc_info=True)
            failed += 1

        except requests.exceptions.RequestException as e:
            logging.error(f"Request exception for {user.get('emailaddress', 'unknown')}: {e}", exc_info=True)
            failed += 1

    return {"success": success, "failed": failed, "skipped": skipped}





{'success': 0, 'failed': 2, 'skipped': 0}
